In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

from astropy.visualization import astropy_mpl_style, quantity_support

plt.style.use(astropy_mpl_style)
quantity_support()

In [ ]:
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.time import Time

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

In [ ]:
Simbad.add_votable_fields('sptype')
Simbad.add_votable_fields('flux(U)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(B)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(V)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(R)') # add the Vega unit Magnitude 
Simbad.add_votable_fields('flux(I)') # add the Vega unit Magnitude 

In [ ]:
def hhmmss2deg(coord):
    hh,mm,ss = coord.replace(' ',':').split(':')
    hh = float(hh)
    mm = float(mm)
    ss = float(ss)
    H = hh+mm/60.+ss/3600.
    deg = H*360./24.
    return deg
    

In [ ]:
def degmmss2deg(coord):
    deg,mm,ss = coord.replace(' ',':').split(':')
    deg = float(deg)
    mm = float(mm)
    ss = float(ss)
    if deg>=0.:
        Deg = deg+mm/60.+ss/3600.
    else:
        Deg = deg-mm/60.-ss/3600.
    return Deg
    

In [ ]:
spectral_types = ['O','B','A','F','G','K','M']

In [ ]:
file_label = 'FIELD_STARS_ACROSS_THE_H-R_DIAGRAM_Type_{0}.csv'

In [ ]:
pre_targets = []

for spec_type_ in spectral_types:
    file_ = file_label.format(spec_type_)
    targets_file_ = open(os.path.join('spectral_atlases/',file_),'r').readlines()
    
    for line in targets_file_[1:]:
        target_ = line.split(',')[0]
        #print(target_)
        pre_targets.append(target_.replace(' ',''))

In [ ]:
print('Number of initial targets = ', len(pre_targets))

In [ ]:
Simbad.query_object(pre_targets[0])

In [ ]:
Simbad.query_object('HD2811')

In [ ]:
pre_target_dict = {}
print('Target name','  RA  ','  DEC  ','  B  ','  V  ','  B-V  ')
for target_ in pre_targets:
    target_ = target_.replace('-',' ')
    try:
        target_info_ = Simbad.query_object(target_)
        #print(target_info_)
        pre_target_dict[target_] = {}
        ra_ = str(target_info_['RA'][0]).replace(' ',':')
        dec_ = str(target_info_['DEC'][0]).replace(' ',':')
        b_ = float(target_info_['FLUX_B'][0])
        v_ = float(target_info_['FLUX_V'][0])
        r_ = float(target_info_['FLUX_R'][0])
        i_ = float(target_info_['FLUX_I'][0])
        b_v_ = b_-v_
        r_i_ = r_-i_

        print(target_,ra_,dec_,b_,v_,r_,i_,b_v_,r_i_)
        add_ = True
    except:
        #print(target_)
        add_ = False
        
    if add_:
        pre_target_dict[target_]['ra'] = ra_
        pre_target_dict[target_]['dec'] = dec_
        pre_target_dict[target_]['V'] = v_
        pre_target_dict[target_]['B'] = b_
        pre_target_dict[target_]['R'] = r_
        pre_target_dict[target_]['I'] = i_
        pre_target_dict[target_]['B_V'] = b_v_
        pre_target_dict[target_]['R_I'] = r_i_
    

### Remove targets with no information about V magnitude or V magnitude greater than limit 

In [ ]:
mag_lim1 = 6.0
mag_lim2 = 3.0

In [ ]:
target_dict = {}
for target_ in list(pre_target_dict.keys()):
    if len(pre_target_dict[target_].keys())!=0:
        if np.isnan(pre_target_dict[target_]['V']) or pre_target_dict[target_]['V']>mag_lim1 or pre_target_dict[target_]['V']<mag_lim2 or type(pre_target_dict[target_]['ra'])==np.ma.core.MaskedConstant:
            pass
        else:
            target_dict[target_] = pre_target_dict[target_]
targets = list(target_dict.keys())
print('Number of targets with {0} < V < {1} = {2}'.format(mag_lim2,mag_lim1,len(targets)))

# Set LSST's location 

In [ ]:
observatory = 'Cerro Pachon'
lsst_loc = EarthLocation.of_site(observatory)

In [ ]:
lsst_loc

In [ ]:
longitude=lsst_loc.lon.degree
latitude=lsst_loc.lat.degree
altitude=lsst_loc.height
print(longitude,latitude,altitude)

# Find time zone to which LSST belongs 

In [ ]:
import timezonefinder
tf = timezonefinder.TimezoneFinder()
lsst_timezone_name = tf.certain_timezone_at(lat=latitude, lng=longitude)

In [ ]:
print(f"Time zone at Rubin LSST Observatory: {lsst_timezone_name}")

# Find time offset with respect to UTC time 

In [ ]:
from datetime import datetime
import pytz

Create time zone object for LSST's coordinates 

In [ ]:
lsst_timezone = pytz.timezone(lsst_timezone_name)

In [ ]:
lsst_timezone

Find UTC offset for LSST's coordinates at present time 

In [ ]:
lsst_time_now = datetime.now(lsst_timezone)
lsst_utcoffset = lsst_time_now.utcoffset().total_seconds()/60/60

In [ ]:
lsst_utcoffset

# Observation date at site 

In [ ]:
DAY = 13
MONTH = 1
YEAR = 2025

In [ ]:
outdir = './targets_{0}-{1:02d}-{2:02d}'.format(YEAR,MONTH,DAY)
if os.path.exists(outdir)==False:
    os.mkdir(outdir)

# Define times with respect to UTC given the UTC offset 

## Set midnight at LSST's place 

Get local midnight time in UTC (i.e. taking into account offset with respect to UTC time) 

In [ ]:
midnight_str = '{0}-{1}-{2} 23:59:59'.format(YEAR,MONTH,DAY)
midnight_local = Time(midnight_str,location=lsst_loc,precision=2) #The same as midnight_str
midnight_utc = Time(midnight_str,scale='utc',location=lsst_loc,precision=2) - lsst_utcoffset*u.hour

In [ ]:
midnight_local

In [ ]:
midnight_utc

## Get local midnight in sidereal time 

In [ ]:
midnight_sidereal = Time(midnight_str,scale='utc',location=lsst_loc,precision=2).sidereal_time('apparent')

In [ ]:
midnight_sidereal

In [ ]:
midnight_sidereal.hourangle

### We will show object's altitude as a function of the difference of hours with respect to local midnight 

In [ ]:
ntimes = 1000

In [ ]:
delta_midnight = np.linspace(-12,12,ntimes)*u.hour
delta_midnight[:5]

In [ ]:
alt_frame = AltAz(obstime=midnight_utc+delta_midnight,location=lsst_loc)

In [ ]:
sidereal_times = (midnight_utc+delta_midnight).sidereal_time('apparent')
len(sidereal_times)

### Get Sun and Moon positions for LSST's frame 

In [ ]:
from astropy.coordinates import get_sun
from astropy.coordinates import get_body

In [ ]:
sun_coords = get_sun(midnight_utc+delta_midnight)
sun_altaz = sun_coords.transform_to(alt_frame)

moon_coords = get_body("moon", midnight_utc+delta_midnight)
moon_altaz = moon_coords.transform_to(alt_frame)

In [ ]:
alt_sun_deg = sun_altaz.alt
alt_moon_deg = moon_altaz.alt

In [ ]:
len(alt_sun_deg)

In [ ]:
sun_max_alt = -18*u.deg
min_ra = sidereal_times.deg[alt_sun_deg<sun_max_alt][0]
max_ra = sidereal_times.deg[alt_sun_deg<sun_max_alt][-1]
min_ra,max_ra

In [ ]:
lsst_loc.lat

### Remove targets with RA,DEC and elevation out of limits 

In [ ]:
min_dec = -55.
max_dec = -5.
max_alt = 80.*u.deg

In [ ]:
type(target_dict[target_]['ra'])=='str'

In [ ]:
final_target_dict = {}
target_coords = {}
for target_ in targets:
    #print(target_)
    #if type(target_dict[target_]['ra'])=='str':
    ra_ = hhmmss2deg(target_dict[target_]['ra'])
    #else:
    #    ra_ = target_dict[target_]['ra']
    #if type(target_dict[target_]['dec'])=='str':
    dec_ = degmmss2deg(target_dict[target_]['dec'])
    #else:
    #    dec_ = target_dict[target_]['dec']
    
    if dec_>=min_dec and dec_<=max_dec and ra_>=min_ra and ra_<=max_ra:
        coords_ = SkyCoord(ra_,dec_,unit="deg")
        if np.max(coords_.transform_to(alt_frame).alt)<max_alt:
            target_coords[target_] = coords_
            final_target_dict[target_] = target_dict[target_]
    
final_targets = list(final_target_dict.keys())

In [ ]:
print('Number of remaining targets = ', len(final_targets))

In [ ]:
target_alts = {}
for target_ in final_targets:
    alt_ = target_coords[target_].transform_to(alt_frame)
    target_alts[target_] = alt_.alt

# Plot altitude evolution over night 

Set hour intervals in UTC and sidereal time 

In [ ]:
delta_ticks = (np.arange(13)*2-12)
sidereal_ticks = (midnight_utc+delta_ticks*u.hour).sidereal_time('apparent')
print(delta_ticks)
print(sidereal_ticks)

local_ticks = []
utc_ticks = []
for dt_ in delta_ticks:
    #print(dt_)
    if dt_+24<=24:
        local_time_ = dt_+24
    else:
        local_time_ = dt_
    local_ticks.append(local_time_)
    
    if dt_+24+-lsst_utcoffset<=24:
        utc_time_ = dt_+24-lsst_utcoffset
    else:
        utc_time_ = dt_-lsst_utcoffset
    utc_ticks.append(utc_time_)
    
local_ticks = np.array(local_ticks)*u.hour
utc_ticks = np.array(utc_ticks)*u.hour
print(local_ticks)
print(utc_ticks)

In [ ]:
plt.style.use('tableau-colorblind10')
cmap = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=len(final_targets))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap)
all_colors = scalarMap.to_rgba(np.arange(len(final_targets)), alpha=1)
color_dict = {}
for i,target_ in enumerate(final_targets):
    color_dict[target_] = all_colors[i]

In [ ]:

fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)

for i,target_ in enumerate(final_targets):
    mag_ = final_target_dict[target_]['V']
    color_ = final_target_dict[target_]['B_V']
    ax.plot(delta_midnight,target_alts[target_],ls='-',color=color_dict[target_],label='{0}, V={1:.2f}, B-V={2:.2f}'.format(target_,mag_,color_))

ax.plot(delta_midnight,alt_sun_deg,ls='--',color='orange',label='Sun')
ax.plot(delta_midnight,alt_moon_deg,ls=':',color='darkgrey',label='Moon')
ax.set_xticks(delta_ticks)
ax.set_xticklabels(labels=local_ticks,rotation=25)
ax.set_xlim(-12*u.hour,12*u.hour)
ax.set_ylim(0*u.degree,90*u.degree)
ax.set_xlabel('{0} local time'.format(lsst_timezone))
ax.set_title('Observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR))
#ax.legend(loc="upper right")


ax2 = ax.twiny()
ax2.plot([],[],ls='')
xticks = ax.get_xticks()
ax2.set_xticks(xticks)
ax2.set_xticklabels(labels=sidereal_ticks,ha='left',rotation=45)
ax2.set_xlim(-12*u.hour,12*u.hour)

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.15
ax3.spines["bottom"].set_position(("axes", offset))
ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)

ax3.set_xticks(xticks)
ax3.set_xticklabels(labels=utc_ticks,rotation=25)
ax3.set_xlabel('UTC time')


plt.tight_layout()

ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -18*u.deg, color='k', zorder=0)

In [ ]:
midnight_utc

# Separate blue and red stars 

In [ ]:
blue_targets = []
red_targets = []
B_V = []

for target_ in final_targets:
    color_ =  final_target_dict[target_]['B']-final_target_dict[target_]['V']
    B_V.append(color_)
    if color_<=0.:
        blue_targets.append(target_)
    else:
        red_targets.append(target_)
B_V = np.array(B_V)

In [ ]:
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)

i = 1
for target_ in blue_targets:
    mag_ = final_target_dict[target_]['V']
    color_ = final_target_dict[target_]['B_V']
    if np.max(target_alts[target_])<max_alt:
        ax.plot(delta_midnight,target_alts[target_],ls='-',color=color_dict[target_],label='{0}) {1}, V={2:.2f}, B-V={3:.2f}'.format(i,target_,mag_,color_))
        ax.text(delta_midnight[target_alts[target_]==np.max(target_alts[target_])],np.max(target_alts[target_])+2*u.deg,i,fontsize=14,color=color_dict[target_])
        i+=1

ax.plot(delta_midnight,alt_sun_deg,ls='--',color='orange',label='Sun')
ax.plot(delta_midnight,alt_moon_deg,ls=':',color='darkgrey',label='Moon')
ax.set_xticks(delta_ticks)
ax.set_xticklabels(labels=local_ticks,rotation=25)
ax.set_xlim(-12*u.hour,12*u.hour)
ax.set_ylim(0*u.degree,90*u.degree)
ax.set_xlabel('{0} local time'.format(lsst_timezone))
ax.set_title('Observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR))
ax.legend(loc="lower right")


ax2 = ax.twiny()
ax2.plot([],[],ls='')
xticks = ax.get_xticks()
ax2.set_xticks(xticks)
ax2.set_xticklabels(labels=sidereal_ticks,ha='left',rotation=45)
ax2.set_xlim(-12*u.hour,12*u.hour)

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.15
ax3.spines["bottom"].set_position(("axes", offset))
ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)

ax3.set_xticks(xticks)
ax3.set_xticklabels(labels=utc_ticks,rotation=25)
ax3.set_xlabel('UTC time')


plt.tight_layout()

ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -18*u.deg, color='k', zorder=0)

In [ ]:
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)

i = 1
for target_ in red_targets:
    mag_ = final_target_dict[target_]['V']
    color_ = final_target_dict[target_]['B_V']
    if np.max(target_alts[target_])<max_alt:
        ax.plot(delta_midnight,target_alts[target_],ls='-',color=color_dict[target_],label='{0}) {1}, V={2:.2f}, B-V={3:.2f}'.format(i,target_,mag_,color_))
        ax.text(delta_midnight[target_alts[target_]==np.max(target_alts[target_])],np.max(target_alts[target_])+2*u.deg,i,fontsize=14,color=color_dict[target_])
        i+=1

ax.plot(delta_midnight,alt_sun_deg,ls='--',color='orange',label='Sun')
ax.plot(delta_midnight,alt_moon_deg,ls=':',color='darkgrey',label='Moon')
ax.set_xticks(delta_ticks)
ax.set_xticklabels(labels=local_ticks,rotation=25)
ax.set_xlim(-12*u.hour,12*u.hour)
ax.set_ylim(0*u.degree,90*u.degree)
ax.set_xlabel('{0} local time'.format(lsst_timezone))
ax.set_title('Observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR))
ax.legend(loc="lower right")


ax2 = ax.twiny()
ax2.plot([],[],ls='')
xticks = ax.get_xticks()
ax2.set_xticks(xticks)
ax2.set_xticklabels(labels=sidereal_ticks,ha='left',rotation=45)
ax2.set_xlim(-12*u.hour,12*u.hour)

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.15
ax3.spines["bottom"].set_position(("axes", offset))
ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)

ax3.set_xticks(xticks)
ax3.set_xticklabels(labels=utc_ticks,rotation=25)
ax3.set_xlabel('UTC time')


plt.tight_layout()

ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -18*u.deg, color='k', zorder=0)

In [ ]:
Nredest = 7
redest_targets = np.array(final_targets)[B_V.argsort()[::-1]][:Nredest]
print(redest_targets)
print(B_V[B_V.argsort()[::-1]][:Nredest])

In [ ]:
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)

i = 1
for target_ in redest_targets:
    mag_ = final_target_dict[target_]['V']
    color_ = final_target_dict[target_]['B_V']
    if np.max(target_alts[target_])<max_alt:
        ax.plot(delta_midnight,target_alts[target_],ls='-',color=color_dict[target_],label='{0}) {1}, V={2:.2f}, B-V={3:.2f}'.format(i,target_,mag_,color_))
        ax.text(delta_midnight[target_alts[target_]==np.max(target_alts[target_])],np.max(target_alts[target_])+2*u.deg,i,fontsize=14,color=color_dict[target_])
        i+=1

ax.plot(delta_midnight,alt_sun_deg,ls='--',color='orange',label='Sun')
ax.plot(delta_midnight,alt_moon_deg,ls=':',color='darkgrey',label='Moon')
ax.set_xticks(delta_ticks)
ax.set_xticklabels(labels=local_ticks,rotation=25)
ax.set_xlim(-12*u.hour,12*u.hour)
ax.set_ylim(0*u.degree,90*u.degree)
ax.set_xlabel('{0} local time'.format(lsst_timezone))
ax.set_title('Observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR))
ax.legend(loc="lower right")


ax2 = ax.twiny()
ax2.plot([],[],ls='')
xticks = ax.get_xticks()
ax2.set_xticks(xticks)
ax2.set_xticklabels(labels=sidereal_ticks,ha='left',rotation=45)
ax2.set_xlim(-12*u.hour,12*u.hour)

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.15
ax3.spines["bottom"].set_position(("axes", offset))
ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)

ax3.set_xticks(xticks)
ax3.set_xticklabels(labels=utc_ticks,rotation=25)
ax3.set_xlabel('UTC time')


plt.tight_layout()

ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -18*u.deg, color='k', zorder=0)

In [ ]:
#target_selection = ['HD92055','HD123934','HD138716','HD156274','HD196171']
#target_selection = ['HD89736','HD123934','HD138716','HD156274','HD109931','HD189340','HD121416']
#target_selection = ['HD109931','HD121416','HD146233','HD138716','HD156274','HD189340','HD212320']
target_selection = redest_targets

### Include CALSPEC targets 

In [ ]:
calspec_targets = ['HD34816']

In [ ]:
target_selection = list(target_selection)+calspec_targets
#target_selection.sort()
target_selection,len(target_selection)

In [ ]:
print('Target name','  RA  ','  DEC  ','  B  ','  V  ','  B-V  ')
for target_ in calspec_targets:
    target_ = target_.replace('-',' ')
    try:
        target_info_ = Simbad.query_object(target_)
        print(target_info_)
        final_target_dict[target_] = {}
        ra_ = str(target_info_['RA'][0]).replace(' ',':')
        ra_ = hhmmss2deg(ra_)
        dec_ = str(target_info_['DEC'][0]).replace(' ',':')
        dec_ = degmmss2deg(dec_)
        b_ = float(target_info_['FLUX_B'][0])
        v_ = float(target_info_['FLUX_V'][0])
        r_ = float(target_info_['FLUX_R'][0])
        i_ = float(target_info_['FLUX_I'][0])
        b_v_ = b_-v_
        r_i_ = r_-i_

        print(target_,ra_,dec_,b_,v_,r_,i_,b_v_,r_i_)
        add_ = True
    except:
        print(target_)
        add_ = False
        
    if add_:
        final_target_dict[target_]['ra'] = ra_
        final_target_dict[target_]['dec'] = dec_
        final_target_dict[target_]['V'] = v_
        final_target_dict[target_]['B'] = b_
        final_target_dict[target_]['R'] = r_
        final_target_dict[target_]['I'] = i_
        final_target_dict[target_]['B_V'] = b_v_
        final_target_dict[target_]['R_I'] = r_i_
        
        #ra_ = hhmmss2deg(str(ra_))
        #dec_ = degmmss2deg(str(dec_))
        if dec_>=min_dec and dec_<=max_dec and ra_>=min_ra and ra_<=max_ra:
            coords_ = SkyCoord(ra_,dec_,unit="deg")
            target_coords[target_] = coords_
            
            alt_ = coords_.transform_to(alt_frame)
            target_alts[target_] = alt_.alt
            

In [ ]:
cmap_final = plt.get_cmap('jet')
cNorm_final = colors.Normalize(vmin=0, vmax=len(target_selection))
scalarMap_final = cmx.ScalarMappable(norm=cNorm_final, cmap=cmap_final)
all_colors_final = scalarMap_final.to_rgba(np.arange(len(target_selection)), alpha=1)
color_dict_final = {}
for i,target_ in enumerate(target_selection):
    color_dict_final[target_] = all_colors_final[i]

In [ ]:
fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(111)

i = 1
for target_ in target_selection:
    mag_ = final_target_dict[target_]['V']
    color_ = final_target_dict[target_]['B_V']
    if target_ in calspec_targets:
        ls_ = '-'
    else:
        ls_ = '--'
    if np.max(target_alts[target_])<max_alt:
        ax.plot(delta_midnight,target_alts[target_],ls=ls_,color=color_dict_final[target_],label='{0}) {1}, V={2:.2f}, B-V={3:.2f}'.format(i,target_,mag_,color_))
        ax.text(delta_midnight[target_alts[target_]==np.max(target_alts[target_])],np.max(target_alts[target_])+2*u.deg,i,fontsize=18,color=color_dict_final[target_])
        i+=1
        
ax.plot(delta_midnight,alt_sun_deg,ls=':',color='orange',label='Sun')
ax.plot(delta_midnight,alt_moon_deg,ls=':',color='darkgrey',label='Moon')
ax.set_xticks(delta_ticks)
ax.set_xticklabels(labels=local_ticks,rotation=25,fontsize=14)
ax.set_xlim(-12*u.hour,12*u.hour)
ax.set_ylim(0*u.degree,90*u.degree)
ax.set_xlabel('{0} local time'.format(lsst_timezone),fontsize=18)
ax.set_ylabel('Elevation [deg]',fontsize=18)
ax.set_title('AuxTel observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR),fontsize=24)
ax.legend(loc="lower right")


ax2 = ax.twiny()
ax2.plot([],[],ls='')
xticks = ax.get_xticks()
ax2.set_xticks(xticks)
ax2.set_xticklabels(labels=sidereal_ticks,ha='left',rotation=45,fontsize=14)
ax2.set_xlim(-12*u.hour,12*u.hour)
ax2.set_xlabel('Local sidereal time',fontsize=18)

ax3 = ax.twiny()
# Move twinned axis ticks and label from top to bottom
ax3.xaxis.set_ticks_position("bottom")
ax3.xaxis.set_label_position("bottom")
# Offset the twin axis below the host
offset = -0.15
ax3.spines["bottom"].set_position(("axes", offset))
ax3.set_frame_on(True)
ax3.patch.set_visible(False) # mandatory
for sp in ax3.spines.values():
    sp.set_visible(False)
ax3.spines["bottom"].set_visible(True)

ax3.set_xticks(xticks)
ax3.set_xticklabels(labels=utc_ticks,rotation=25,fontsize=14)
ax3.set_xlabel('UTC time', fontsize=18)

ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -0*u.deg, color='0.5', zorder=0)
ax.fill_between(delta_midnight, 0*u.deg, 90*u.deg,
                 alt_sun_deg < -18*u.deg, color='k', zorder=0)

plt.tight_layout()
plt.savefig(os.path.join(outdir,'targets_{0}-{1}-{2}_alt-time.png'.format(DAY,MONTH,YEAR)))

In [ ]:
def makePolarPlot(azimuthsInDegrees, zenithAngles, marker=".",ax=None,
                       title=None, color=None, objName=None,lw=0.5):
    if ax==None:
        _ = plt.figure(figsize=(20, 10))
        ax = plt.subplot(111, polar=True)
        
    ax.plot([a*np.pi/180 for a in azimuthsInDegrees], zenithAngles, marker, c=color, label=objName,lw=lw)
    if title:
        ax.set_title(title, va='bottom')
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.set_rlim(0, 90)
    return ax

In [ ]:
darktime_mask = alt_sun_deg < -18*u.deg

In [ ]:
fig = plt.figure(figsize=(10, 10))

ax = fig.add_subplot(111, polar=True)

# compute zenith and azimuth angles
az_moon_deg = moon_altaz.az.deg
zen_moon_deg = moon_altaz.zen.deg

az_moon_deg_night = az_moon_deg[darktime_mask]
zen_moon_deg_night = zen_moon_deg[darktime_mask]
ax=makePolarPlot(az_moon_deg_night, zen_moon_deg_night,ax=ax ,marker=".-",title=None, color = "darkgrey",objName="Moon",lw=8)


for target_ in target_selection:
    
    target_altaz_ = target_coords[target_].transform_to(alt_frame)
    az_target_deg_ = target_altaz_.az.deg
    zen_target_deg_ = target_altaz_.zen.deg
    
    az_target_deg_night_ = az_target_deg_[darktime_mask]
    zen_target_deg_night_ = zen_target_deg_[darktime_mask]
    
    
    ax=makePolarPlot(az_target_deg_, zen_target_deg_,ax=ax, marker="--",title=None, color=color_dict_final[target_], lw=0.5)
    ax=makePolarPlot(az_target_deg_night_, zen_target_deg_night_,ax=ax ,marker="*-",title=None, color=color_dict_final[target_],objName=target_,lw=0.5)
    

ax.legend(bbox_to_anchor=(1.08, 1), prop={'size': 15}, loc='upper left')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_rlim(0, 90)
ax.set_xticklabels(['N', '', 'E', '', 'S', '', 'W', ''])
ax.set_title('AuxTel observing run of {0}-{1}-{2}'.format(DAY,MONTH,YEAR),fontsize=18)
plt.savefig(os.path.join(outdir,'targets_{0}-{1}-{2}_az-zen.png'.format(DAY,MONTH,YEAR)))

# Exposure time calculations 

In [ ]:
(3000-1390)/2.69

In [ ]:
# lambda_eff in units of pixel 
pix_factor = 2.69
pix0 = 1390
lambda_eff_b = int(445*pix_factor)+pix0
lambda_eff_v = int(551*pix_factor)+pix0
lambda_eff_r = int(658*pix_factor)+pix0
lambda_eff_i = int(806*pix_factor)+pix0
lambda_eff_b,lambda_eff_v,lambda_eff_r,lambda_eff_i

In [ ]:
def calc_int_adu(mag1,mag2,F1):
    F2 = F1*10.**((mag1-mag2)/2.5)
    return F2

In [ ]:
lambda_eff_v

In [ ]:
t_exp_ref = 30.
v_mag_ref = 7.5
int_adu_v_ref = 1149531.8699951172
num_pix_v = 473

In [ ]:
saturation = 100000 #ADU/pix 
goal_adu = 10000 #ADU/pixel
t_exp = 30. #s
Nposs = 17

In [ ]:
file1 = open(os.path.join(outdir,'target_exptime_{0}-{1}-{2}.txt'.format(DAY,MONTH,YEAR)),'w')
string0 = 'Target    V \t B-V \t Int./pix (30s)  Int./pix/s \t Meets goal \t Saturates \t N_exp ({0}s) \t Total time \n'.format(t_exp)
print(string0)
file1.write(string0)
for target_ in target_selection:
    adu_pix_ = calc_int_adu(v_mag_ref,final_target_dict[target_]['V'],int_adu_v_ref)/num_pix_v
    #adu_pix_ = np.round(adu_pix_)
    v_ = np.round(final_target_dict[target_]['V'],2)
    b_v_ = np.round(final_target_dict[target_]['B_V'],2)
    adu_pix_s_ = adu_pix_/t_exp_ref 

    good_ = adu_pix_>=goal_adu
    saturates_ = adu_pix_>=saturation
    
    Nexp_ = int(np.round(saturation/adu_pix_,0))
    total_time_ = Nexp_*t_exp*Nposs
    
    string_ = '{0}   {1:.2f} \t {2:.2f} \t {3:.2f} \t {4:.2f} \t {5} \t \t {6} \t \t {7} \t \t {8:.2f}s \n'.format(target_,v_,b_v_,adu_pix_,adu_pix_s_,good_,saturates_,Nexp_,total_time_)
    print(string_)
    file1.write(string_)
file1.close()

In [ ]:
string0 = 'Target \t ADU/pix (30s) \t t_exp (s) \t N_exp \t Total time (s) \n'.format(t_exp)
print(string0)
for target_ in target_selection:
    adu_pix_ = int(np.round(calc_int_adu(v_mag_ref,final_target_dict[target_]['V'],int_adu_v_ref)/num_pix_v,0))
    
    Nexp_ = int(np.round(saturation/adu_pix_,0))
    total_time_ = Nexp_*int(t_exp)*Nposs
    
    string_ = '{0} \t {1} \t {2} \t {3} \t {4} \n'.format(target_,adu_pix_,int(t_exp),Nexp_,total_time_)
    print(string_)
    


In [ ]:
file2 = open(os.path.join(outdir,'target_coords_{0}-{1}-{2}.txt'.format(DAY,MONTH,YEAR)),'w')
string0 = 'Target \t  RA \t \t  DEC \t \t   V \t  B-V \n'
print(string0)
file2.write(string0)
for target_ in target_selection:
    h_,m_,s_ = target_coords[target_].ra.hms
    ra_ = '{0:02d}:{1}:{2:.4f}'.format(int(h_),int(m_),s_)
    d_,m_,s_ = target_coords[target_].dec.dms
    if d_<0.:
        d_ = '-{0:02d}'.format(np.abs(int(d_)))
    else:
        d_ = '{0:02d}'.format(int(d_))
    if np.abs(s_)<10.:
        dec_ = '{0}:{1}:0{2:.4f}'.format(d_,int(np.abs(m_)),np.abs(s_))
    else:
        dec_ = '{0}:{1}:{2:.4f}'.format(d_,int(np.abs(m_)),np.abs(s_))
    v_ = np.round(final_target_dict[target_]['V'],2)
    b_v_ = np.round(final_target_dict[target_]['B_V'],2)
    
    string_ = '{0}   {1}   {2}   {3:.2f}   {4:.2f} \n'.format(target_,ra_,dec_,v_,b_v_)
    print(string_)
    file2.write(string_)
file2.close()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

r = 1

alts = np.arange(0.,90.,10)*np.pi/180.
azs = np.arange(0.,360.,1)*np.pi/180.

for alt_ in alts:
    x_ = r*np.cos(azs)*np.cos(alt_)
    y_ = r*np.sin(azs)*np.cos(alt_)
    z_ = r*np.sin(alt_)
    ax.plot(x_,y_,z_,ls='-',color='grey',lw='0.1')
    #ax.plot(x_,y_)

alts = np.arange(0.,90.,1)*np.pi/180.
azs = np.arange(0.,360.,20)*np.pi/180.

for az_ in azs:
    x_ = r*np.cos(az_)*np.cos(alts)
    y_ = r*np.sin(az_)*np.cos(alts)
    z_ = r*np.sin(alts)
    ax.plot(x_,y_,z_,ls='-',color='grey',lw='0.1')
    #ax.plot(x_,y_)


# compute zenith and azimuth angles
az_moon = moon_altaz.az.radian
alt_moon = moon_altaz.alt.radian

az_moon_night = az_moon[darktime_mask]
alt_moon_night = alt_moon[darktime_mask]

alt_mask = alt_moon_night>=0.

az_moon_vis = az_moon_night[alt_mask]
alt_moon_vis = alt_moon_night[alt_mask]

xmoon = r*np.cos(az_moon_vis)*np.cos(alt_moon_vis)
ymoon = r*np.sin(az_moon_vis)*np.cos(alt_moon_vis)
zmoon = r*np.sin(alt_moon_vis)

ax.plot(xmoon,ymoon,zmoon,ls='-',marker='',color='darkgrey',label='Moon',lw=8)

for target_ in target_selection:
    
    target_altaz_ = target_coords[target_].transform_to(alt_frame)
    az_target_ = target_altaz_.az.radian
    alt_target_ = target_altaz_.alt.radian
    
    az_target_night_ = az_target_[darktime_mask]
    alt_target_night_ = alt_target_[darktime_mask]

    alt_mask_ = alt_target_night_>=0.
    
    az_target_vis_ = az_target_night_[alt_mask_]
    alt_target_vis_ = alt_target_night_[alt_mask_]
    
    x_ = r*np.cos(az_target_vis_)*np.cos(alt_target_vis_)
    y_ = r*np.sin(az_target_vis_)*np.cos(alt_target_vis_)
    z_ = r*np.sin(alt_target_vis_)
    
    ax.plot(x_,y_,z_,ls='-',marker="*",color=color_dict_final[target_],lw=0.25,label=target_)
    #ax=makePolarPlot(az_target_deg_, zen_target_deg_,ax=ax, marker="--",title=None, color=color_dict_final[target_], lw=0.5)
    #ax=makePolarPlot(az_target_deg_night_, zen_target_deg_night_,ax=ax ,marker="*-",title=None, color=color_dict_final[target_],objName=target_,lw=0.5)
    

ax.set_xlim(-1.,1.)
ax.set_ylim(-1.,1.)
ax.set_zlim(0.,)
ax.legend(loc="best",fontsize=10)


In [ ]:
from ipyaladin import Aladin
from sidecar import Sidecar

In [ ]:
from astropy.coordinates import Angle

In [ ]:
aladin = Aladin()
with Sidecar(title="aladin_output"):
    display(aladin)

In [ ]:
aladin.fov = Angle(6.3 * u.arcmin)

In [ ]:
print(len(target_selection))

In [ ]:
view_target = 0
aladin.target = target_coords[target_selection[view_target]]